<a href="https://colab.research.google.com/github/nickprock/appunti_data_science/blob/master/semantic-search/advent-of-haystack/Advent_of_Haystack_Serialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack: Day 10

_Make a copy of this Colab to start_

For the last challenge of Advent of Haystack, we are asking you to create your own _marshaller_ to use a pipeline that was seriealized with msgpack.

Your task is to complete **Steps 2 onwards**

## 1) Install Dependencies

In [ ]:
!pip install haystack-ai msgpack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### Enabling Telemetry

Knowing you’re running this challenge helps us know whether Advent of Haystack is helping people learn about Haystack 2.0-Beta. But you can always opt out by commenting the following line.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running("challenge_10")

When we de-serialize a pipeline, all the components are automatically loaded
so we can't pass the api key to the PromptBuilder constructor as usual. The
best practice is to let the PromptBuilder instance access the special
environment variable `OPENAI_API_KEY` that we set here.

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI Api key: ")

Enter OpenAI Api key: ··········


## 2) Create a custom Marshaller


Documentation page for [serialization](https://docs.haystack.deepset.ai/v2.0/docs/serialization)

We will use the custom `MsgpackMarshaller` to be able to use the pipeline degined in `ancient_instructions`

In [ ]:
# MessagePack is an efficient binary serialization format. It's not supposed to be human-readable, as you can see:
ancient_instructions = b'\x84\xa8metadata\x80\xb1max_loops_allowedd\xaacomponents\x86\xa9converter\x82\xa4type\xd92haystack.components.converters.html.HTMLToDocument\xafinit_parameters\x80\xa7fetcher\x82\xa4type\xd9<haystack.components.fetchers.link_content.LinkContentFetcher\xafinit_parameters\x84\xb0raise_on_failure\xc3\xabuser_agents\x91\xd9#haystack/LinkContentFetcher/0.152.0\xaeretry_attempts\x02\xa7timeout\x03\xa3llm\x82\xa4type\xd92haystack.components.generators.openai.GPTGenerator\xafinit_parameters\x85\xaamodel_name\xadgpt-3.5-turbo\xb2streaming_callback\xc0\xacapi_base_url\xb9https://api.openai.com/v1\xb1generation_kwargs\x80\xadsystem_prompt\xc0\xaeprompt_builder\x82\xa4type\xd99haystack.components.builders.prompt_builder.PromptBuilder\xafinit_parameters\x81\xa8template\xd9\x8a Acc  ding to these docu  nts:\n{% for  oc in documents %}  {{ doc.con     }} {% endfor %}\nAnswer the given qu  tion: {{question}} Answer: \xa6ranker\x82\xa4type\xd9Phaystack.components.rankers.transformers_similarity.TransformersSimilarityRanker\xafinit_parameters\x84\xa6device\xa3cpu\xb2model_name_or_path\xd9$cross-encoder/ms-marco-MiniLM-L-6-v2\xa5token\xc0\xa5top_k\x03\xa8splitter\x82\xa4type\xd9Dhaystack.components.preprocessors.document_splitter.DocumentSplitter\xafinit_parameters\x83\xa8split_by\xa4word\xacsplit_length2\xadsplit_overlap\x00\xabconnections\x95\x82\xa6sender\xb3converter.documents\xa8receiver\xb2splitter.documents\x82\xa6sender\xaffetcher.streams\xa8receiver\xb1converter.sources\x82\xa6sender\xb5prompt_builder.prompt\xa8receiver\xaallm.prompt\x82\xa6sender\xb0ranker.documents\xa8receiver\xb8prompt_builder.documents\x82\xa6sender\xb2splitter.documents\xa8receiver\xb0ranker.documents'

In [ ]:
import msgpack


class MsgpackMarshaller:
    """
    Custom Messagepack marshaller implementing
    the Marshaller protocol in Haystack.
    """

    def unmarshal(self, data_):
        return dict(msgpack.unpackb(data_, use_list=False, raw=False))


In [ ]:
test = MsgpackMarshaller()

In [ ]:
mar = test.unmarshal(ancient_instructions)

In [ ]:
from typing import Dict, Any, Union

import yaml


class YamlMarshaller:
    def marshal(self, dict_: Dict[str, Any]) -> str:
        return yaml.dump(dict_)

    def unmarshal(self, data_: Union[str, bytes, bytearray]) -> Dict[str, Any]:
        return yaml.safe_load(data_)

In [ ]:
yaml_mar = YamlMarshaller().marshal(mar)

In [ ]:
yaml_mar

"components:\n  converter:\n    init_parameters: {}\n    type: haystack.components.converters.html.HTMLToDocument\n  fetcher:\n    init_parameters:\n      raise_on_failure: true\n      retry_attempts: 2\n      timeout: 3\n      user_agents: !!python/tuple\n      - haystack/LinkContentFetcher/0.152.0\n    type: haystack.components.fetchers.link_content.LinkContentFetcher\n  llm:\n    init_parameters:\n      api_base_url: https://api.openai.com/v1\n      generation_kwargs: {}\n      model_name: gpt-3.5-turbo\n      streaming_callback: null\n      system_prompt: null\n    type: haystack.components.generators.openai.GPTGenerator\n  prompt_builder:\n    init_parameters:\n      template: ' Acc  ding to these docu  nts:\n\n        {% for  oc in documents %}  {{ doc.con     }} {% endfor %}\n\n        Answer the given qu  tion: {{question}} Answer: '\n    type: haystack.components.builders.prompt_builder.PromptBuilder\n  ranker:\n    init_parameters:\n      device: cpu\n      model_name_or_path

## 3) Convert Messagepack to YAML

Read the ancient instructions using the custom marshaller into a `Pipeline` object, then use the default marshaller to convert the pipeline again, this time to YAML.

In [ ]:
from haystack import Pipeline

pipe = Pipeline.loads(yaml_mar)
print(pipe.dumps())

ConstructorError: ignored

# 4) Edit the YAML representation of the pipeline to fix the errors

Contrary to Messagepack, YAML is supposed to be human-readable, and Haystack encourages you to work with pipelines using this format instead of Python whenever it makes sense.

Can you spot the errors in the YAML code you printed in the previous cell?

You can copy over the YAML output from the section above to try to spot the errors.

In [ ]:
broken_pipeline_definition = """
components:
  converter:
    init_parameters: {}
    type: haystack.components.converters.html.HTMLToDocument
  fetcher:
    init_parameters:
      raise_on_failure: true
      retry_attempts: 2
      timeout: 3
      user_agents: !!python/tuple
      - haystack/LinkContentFetcher/0.152.0
    type: haystack.components.fetchers.link_content.LinkContentFetcher
  llm:
    init_parameters:
      api_base_url: https://api.openai.com/v1
      generation_kwargs: {}
      model_name: gpt-3.5-turbo
      streaming_callback: null
      system_prompt: null
    type: haystack.components.generators.openai.GPTGenerator
  prompt_builder:
    init_parameters:
      template: ' Acc  ding to these docu  nts:

        {% for  oc in documents %}  {{ doc.con     }} {% endfor %}

        Answer the given qu  tion: {{question}} Answer: '
    type: haystack.components.builders.prompt_builder.PromptBuilder
  ranker:
    init_parameters:
      device: cpu
      model_name_or_path: cross-encoder/ms-marco-MiniLM-L-6-v2
      token: null
      top_k: 3
    type: haystack.components.rankers.transformers_similarity.TransformersSimilarityRanker
  splitter:
    init_parameters:
      split_by: word
      split_length: 50
      split_overlap: 0
    type: haystack.components.preprocessors.document_splitter.DocumentSplitter
connections: !!python/tuple
- receiver: splitter.documents
  sender: converter.documents
- receiver: converter.sources
  sender: fetcher.streams
- receiver: llm.prompt
  sender: prompt_builder.prompt
- receiver: prompt_builder.documents
  sender: ranker.documents
- receiver: ranker.documents
  sender: splitter.documents
max_loops_allowed: 100
metadata: {}
"""


## 5) Run the fixed pipeline

In [ ]:
fixed_pipeline_definition = """
components:
  converter:
    init_parameters: {}
    type: haystack.components.converters.html.HTMLToDocument
  fetcher:
    init_parameters:
      raise_on_failure: true
      retry_attempts: 2
      timeout: 3
      user_agents:
      - haystack/LinkContentFetcher/0.152.0
    type: haystack.components.fetchers.link_content.LinkContentFetcher
  llm:
    init_parameters:
      api_base_url: https://api.openai.com/v1
      generation_kwargs: {}
      model_name: gpt-3.5-turbo
      streaming_callback: null
      system_prompt: null
    type: haystack.components.generators.openai.GPTGenerator
  prompt_builder:
    init_parameters:
      template: ' Acc  ding to these documents:

        {% for  doc in documents %}  {{ doc.content }} {% endfor %}

        Answer the given question: {{question}} Answer: '
    type: haystack.components.builders.prompt_builder.PromptBuilder
  ranker:
    init_parameters:
      device: cpu
      model_name_or_path: cross-encoder/ms-marco-MiniLM-L-6-v2
      token: null
      top_k: 3
    type: haystack.components.rankers.transformers_similarity.TransformersSimilarityRanker
  splitter:
    init_parameters:
      split_by: word
      split_length: 50
      split_overlap: 0
    type: haystack.components.preprocessors.document_splitter.DocumentSplitter
connections:
- receiver: splitter.documents
  sender: converter.documents
- receiver: converter.sources
  sender: fetcher.streams
- receiver: llm.prompt
  sender: prompt_builder.prompt
- receiver: prompt_builder.documents
  sender: ranker.documents
- receiver: ranker.documents
  sender: splitter.documents
max_loops_allowed: 100
metadata: {}
"""

working_pipeline = Pipeline.loads(fixed_pipeline_definition)
result = working_pipeline.run({
    "prompt_builder": {"question": "how do I start a lathe?"},
    "ranker": {"query": "how do I start a lathe?"},
    "fetcher": {"urls": ["https://en.wikipedia.org/wiki/Lathe"]}
})
print(result["llm"]["replies"][0])

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

To start a lathe, follow these steps:

1. Ensure that the lathe is properly set up on a sturdy workbench or stand.
2. Make sure that all the necessary tools and accessories are in place, such as cutting tools, chuck or drive, center, etc.
3. Check that the lathe is properly plugged in and the power switch is in the "off" position.
4. Put on safety goggles and any other personal protective equipment required.
5. Ensure that the workpiece is securely mounted on the lathe, either between centers or in a chuck.
6. Adjust the tool rest and tool position according to the desired cutting operation.
7. Turn on the lathe by flipping the power switch to the "on" position.
8. Gradually increase the speed of the lathe according to the type of workpiece and cutting operation.
9. Gently engage the cutting tool with the workpiece to start the turning process.
10. Apply consistent, even pressure to the tool while maintaining control and monitoring the cutting operation.
11. Continuously inspect the wo